In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
def get_mysql_engine(host, user, password, database):
    """
    Creates and returns a SQLAlchemy MySQL engine
    """
    return create_engine(
        f"mysql+pymysql://{user}:{password}@{host}/{database}",
        pool_pre_ping=True
    )

In [ ]:
import pyodbc

# Connection details
server = '10.147.17.61'     
database = 'MESinterface'     
username = 'JKBTP_USER'    
password = 'JK@$userbtp'  

# Build connection string
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
)

# Create connection
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

print("Connected successfully!")


In [ ]:
import pandas as pd

def get_o_production_data(component, conn, start_time, end_time):
    """
    Fetch O_Production data for a component
    within a given SyncTime range
    """

    tables_map = {
        'TREAD': ['DualEx', 'Duplex', 'QuintuPlex'],
        'INNER LINER': ['trc', 'TRCNew'],
        'BELT': ['WBC', 'WBCNew'],
        'SIDEWALL': ['TRIPLEX', 'Duplex', 'QuintuPlex']
    }

    if component not in tables_map:
        raise ValueError(f"Invalid component: {component}")

    schemas = tables_map[component]
    df_list = []

    for schema in schemas:
        query = f"""
        SELECT
            [MachineNo],
            [MachineCode],
            [ProductionID],
            [ItemCode],
            [ItemName],
            [TotalQuantity],
            [ScrapQuantity],
            [SyncTime],
            [UserName],
            [MHECode],
            [QualityStatus],
            [LiveQty],
            '{schema}' AS source_schema
        FROM {schema}.O_Production
        WHERE SyncTime BETWEEN ? AND ?
        """

        try:
            df = pd.read_sql(query, conn, params=[start_time, end_time])
            df_list.append(df)
            print(f"✔ Loaded {len(df)} rows from {schema}.O_Production")

        except Exception as e:
            print(f"❌ Error loading {schema}.O_Production → {e}")

    if not df_list:
        return pd.DataFrame()

    final_df = pd.concat(df_list, ignore_index=True)
    print(f"\n✅ Total merged rows: {len(final_df)}")

    return final_df


In [ ]:
import pandas as pd

def get_imaterials(conn, start_time, end_time):
    """
    Loads I_Material data from TBMStage1 and TBMStage2
    for the same dtandTime range and merges them.
    """

    tables = [
        ("TBMStage2", "STAGE2"),
        ("TBMStage1", "STAGE1")
    ]

    df_list = []

    for schema, stage_name in tables:
        query = f"""
        SELECT
            [Lot_Id],
            [UOM],
            [Qty],
            [MaterialCode],
            [LiveQty],
            [dtandTime],
            '{stage_name}' AS stage,
            '{schema}' AS source_schema
        FROM {schema}.I_Material
        WHERE dtandTime BETWEEN ? AND ?
        """

        try:
            df = pd.read_sql(query, conn, params=[start_time, end_time])
            df_list.append(df)
            print(f"✔ Loaded {len(df)} rows from {schema}.I_Material")

        except Exception as e:
            print(f" Error loading {schema}.I_Material → {e}")

    if not df_list:
        return pd.DataFrame()

    final_df = pd.concat(df_list, ignore_index=True)
    print(f"\n✅ Total merged rows: {len(final_df)}")

    return final_df


start_time = "2025-01-01 00:00:00"
end_time   = "2025-01-01 23:59:59"

df_imaterials = get_imaterials(conn, start_time, end_time)


In [ ]:
start_time = "2025-01-01 00:00:00"
end_time   = "2025-01-01 23:59:59"

df_tread = get_o_production_data(
    component="TREAD",
    conn=conn,
    start_time=start_time,
    end_time=end_time
)


In [ ]:
# ==================================================
# LOAD INVENTORY FROM jkplanningV1 (Inventory)
# ==================================================
def load_inventory_mysql(
    start_datetime: str,
    end_datetime: str
) -> pd.DataFrame:
    """
    Loads inventory data for a given time frame.

    Parameters
    ----------
    start_datetime : str
        Format: 'YYYY-MM-DD HH:MM:SS'
    end_datetime : str
        Format: 'YYYY-MM-DD HH:MM:SS'

    Returns
    -------
    pd.DataFrame
        Inventory records within the given time window
    """

    # -----------------------
    # DB CONFIG (READ ONLY)
    # -----------------------
    host = "35.208.174.2"
    user = "root"
    password = "Dev112233"
    database = "jkplanningV1"

    engine = get_mysql_engine(host, user, password, database)

    # -----------------------
    # QUERY
    # -----------------------
    query = """
    SELECT *
    FROM Inventory
    WHERE productionTime BETWEEN %s AND %s
    """

    # -----------------------
    # LOAD DATA
    # -----------------------
    df = pd.read_sql(
        query,
        engine,
        params=[start_datetime, end_datetime]
    )

    print(f"✔ Loaded {len(df)} inventory rows")

    return df


In [ ]:
inventory_df = load_inventory_mysql(
    start_datetime="2025-12-30 05:00:00",
    end_datetime="2025-12-31 06:00:00"
)
